In [1]:
!pip install textblob
!pip install wordcloud
!pip install nltk

     -------------------------------------- 636.8/636.8 kB 2.0 MB/s eta 0:00:00


In [70]:
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [71]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import codecs

In [72]:
df = pd.read_csv('dataset_twitterfinansial.csv',sep=";", encoding='cp1252')

In [73]:
df

,Created-At,From-User,From-User-Id,To-User,To-User-Id,Language,Source,Text,Geo-Location-Latitude,Geo-Location-Longitude,Retweet-Count,Id
0,20/08/2022 11:10,Minor League Baseball,34363347,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,17-RUN INNING!\n\nThe Triple-A @astros affilia...,NaN,NaN,311,"1,56E+18"
1,20/08/2022 08:41,Ebony Elizabeth Thomas,322742783,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,"Ambassador @WBStevens, as an SWFA affiliate me...",NaN,NaN,71,"1,56E+18"
2,20/08/2022 02:50,The Circuit,1342662080,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,The Circuit's 3SSB Coach of the Year: pres. by...,NaN,NaN,37,"1,56E+18"
3,20/08/2022 21:03,A Deal Sharing Community,3881432236,NaN,-1,en,<a href='https://app.socialpilot.co/' rel='nof...,?Multiuse Code?\n50%OFF\nCode: 50VNXFXO\n?Let ...,NaN,NaN,0,"1,56E+18"
4,20/08/2022 21:03,COMMISSIONS OPEN WOO -saben/binx,"1,31E+18",NaN,-1,en,<a href='http://twitter.com/download/iphone' r...,STREAMING TONIGHT!\nfirst stream as affiliate!...,NaN,NaN,0,"1,56E+18"
...,...,...,...,...,...,...,...,...,...,...,...,...
3176,19/08/2022 15:10,Amit (Taylor's version),"1,52E+18",NaN,-1,en,<a href='http://twitter.com/download/android' ...,RT @aneetta_joby_: Money saving hack: Live wit...,NaN,NaN,113,"1,56E+18"
3177,19/08/2022 15:10,BPBT,"1,35E+18",NaN,-1,th,<a href='http://twitter.com/download/android' ...,RT @Koonkonrak: ???????? ??????????? ????????\...,NaN,NaN,267,"1,56E+18"
3178,19/08/2022 15:10,LISA?,"1,49E+18",NaN,-1,en,<a href='http://twitter.com/download/android' ...,RT @chynxxxxx: This lisa is very sexc\n\nFEEL ...,NaN,NaN,66,"1,56E+18"
3179,19/08/2022 15:10,KIRWEZOGA,577753527,NaN,-1,en,<a href='http://twitter.com/download/android' ...,"RT @ChuksDan: @atiku I think, by way of punish...",NaN,NaN,2,"1,56E+18"


In [74]:
df2=df[['Id','Text']]
df2

,Id,Text
0,"1,56E+18",17-RUN INNING!\n\nThe Triple-A @astros affilia...
1,"1,56E+18","Ambassador @WBStevens, as an SWFA affiliate me..."
2,"1,56E+18",The Circuit's 3SSB Coach of the Year: pres. by...
3,"1,56E+18",?Multiuse Code?\n50%OFF\nCode: 50VNXFXO\n?Let ...
4,"1,56E+18",STREAMING TONIGHT!\nfirst stream as affiliate!...
...,...,...
3176,"1,56E+18",RT @aneetta_joby_: Money saving hack: Live wit...
3177,"1,56E+18",RT @Koonkonrak: ???????? ??????????? ????????\...
3178,"1,56E+18",RT @chynxxxxx: This lisa is very sexc\n\nFEEL ...
3179,"1,56E+18","RT @ChuksDan: @atiku I think, by way of punish..."


In [75]:
#--------Case Folding--------
df2['Text'] = df2['Text'].str.lower()

print('Case Folding Result : \n')
print(df2['Text'].head(5))
print('\n\n\n')

Case Folding Result : 

0    17-run inning!\n\nthe triple-a @astros affilia...
1    ambassador @wbstevens, as an swfa affiliate me...
2    the circuit's 3ssb coach of the year: pres. by...
3    ?multiuse code?\n50%off\ncode: 50vnxfxo\n?let ...
4    streaming tonight!\nfirst stream as affiliate!...
Name: Text, dtype: object






C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\1141292331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text'] = df2['Text'].str.lower()


In [76]:
import string 
import re #regex library
import nltk
nltk.download('punkt')
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing & Cleansing  ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove rt sebagai retweet
    text = text.replace("re", "").strip()
    # remove incomplete URL
                
df2['Text'] = df2['Text'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text or '') #1

df2['Text'] = df2['Text'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df2['Text'] = df2['Text'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df2['Text'] = df2['Text'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df2['Text'] = df2['Text'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text or '') #2

df2['Text'] = df2['Text'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df2['Text_tokens'] = df2['Text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df2['Text_tokens'].head(10))
print('\n\n\n')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NIKORAM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\3623354553.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text'] = df2['Text'].apply(remove_tweet_special)
C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\3623354553.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text'] = df2['Text'].apply(remove_number)
C:\Users\NIKORAM\AppData\

Tokenizing Result : 

0    []
1    []
2    []
3    []
4    []
5    []
6    []
7    []
8    []
9    []
Name: Text_tokens, dtype: object






C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\3623354553.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text_tokens'] = df2['Text'].apply(word_tokenize_wrapper)


In [77]:
#Stemming Data
ps = PorterStemmer() 

def stemming_data(x):
    return ps.stem(x)

df['Text'] = df['Text'].apply(stemming_data)

In [78]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword english
list_stopwords = stopwords.words('english')
print(len(list_stopwords))

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df2['Text_tokens_WSW'] = df2['Text_tokens'].apply(stopwords_removal) 

print(df2['Text_tokens_WSW'].head())

179
0    []
1    []
2    []
3    []
4    []
Name: Text_tokens_WSW, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NIKORAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\2087300649.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text_tokens_WSW'] = df2['Text_tokens'].apply(stopwords_removal)


In [79]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)


df2['Text_tokens_fdist'] = df2['Text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(df2['Text_tokens_fdist'].head(10).apply(lambda x : x.most_common()))

Frequency Tokens : 

0    []
1    []
2    []
3    []
4    []
5    []
6    []
7    []
8    []
9    []
Name: Text_tokens_fdist, dtype: object


C:\Users\NIKORAM\AppData\Local\Temp\ipykernel_10208\2633044772.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text_tokens_fdist'] = df2['Text_tokens'].apply(freqDist_wrapper)


In [80]:
dataset = list(df['Text'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(dataset):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0:
        total_positif += 1
        status.append('positive')
    elif analysis.sentiment.polarity == 0:
        total_netral += 1
        status.append('neutral')
    else:
        total_negatif += 1
        status.append('negative')

    total += 1 
    
print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 1051
Netral = 1641
Negatif = 489

Total Data : 3181


In [81]:
status = pd.DataFrame({'klasifikasi': status})
df['klasifikasi'] = status
df

,Created-At,From-User,From-User-Id,To-User,To-User-Id,Language,Source,Text,Geo-Location-Latitude,Geo-Location-Longitude,Retweet-Count,Id,klasifikasi
0,20/08/2022 11:10,Minor League Baseball,34363347,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,17-run inning!\n\nthe triple-a @astros affilia...,NaN,NaN,311,"1,56E+18",neutral
1,20/08/2022 08:41,Ebony Elizabeth Thomas,322742783,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,"ambassador @wbstevens, as an swfa affiliate me...",NaN,NaN,71,"1,56E+18",positive
2,20/08/2022 02:50,The Circuit,1342662080,NaN,-1,en,<a href='https://mobile.twitter.com' rel='nofo...,the circuit's 3ssb coach of the year: pres. by...,NaN,NaN,37,"1,56E+18",positive
3,20/08/2022 21:03,A Deal Sharing Community,3881432236,NaN,-1,en,<a href='https://app.socialpilot.co/' rel='nof...,?multiuse code?\n50%off\ncode: 50vnxfxo\n?let ...,NaN,NaN,0,"1,56E+18",neutral
4,20/08/2022 21:03,COMMISSIONS OPEN WOO -saben/binx,"1,31E+18",NaN,-1,en,<a href='http://twitter.com/download/iphone' r...,streaming tonight!\nfirst stream as affiliate!...,NaN,NaN,0,"1,56E+18",positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3176,19/08/2022 15:10,Amit (Taylor's version),"1,52E+18",NaN,-1,en,<a href='http://twitter.com/download/android' ...,rt @aneetta_joby_: money saving hack: live wit...,NaN,NaN,113,"1,56E+18",positive
3177,19/08/2022 15:10,BPBT,"1,35E+18",NaN,-1,th,<a href='http://twitter.com/download/android' ...,rt @koonkonrak: ???????? ??????????? ????????\...,NaN,NaN,267,"1,56E+18",neutral
3178,19/08/2022 15:10,LISA?,"1,49E+18",NaN,-1,en,<a href='http://twitter.com/download/android' ...,rt @chynxxxxx: this lisa is very sexc\n\nfeel ...,NaN,NaN,66,"1,56E+18",positive
3179,19/08/2022 15:10,KIRWEZOGA,577753527,NaN,-1,en,<a href='http://twitter.com/download/android' ...,"rt @chuksdan: @atiku i think, by way of punish...",NaN,NaN,2,"1,56E+18",positive


In [82]:
df2=df[['Id','Text','klasifikasi','Text_tokens', 'Text_tokens_fdist','Text_tokens_WSW' ]]
df2

KeyError: "['Text_tokens', 'Text_tokens_fdist', 'Text_tokens_WSW'] not in index"

In [ ]:
def show_pie(label, data, legend_title) :
    fig, ax = plt.subplots(figsize=(8, 10), subplot_kw=dict(aspect='equal'))

    labels = [x.split()[-1] for x in label]

    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}% ({:d})".format(pct, absolute)

    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data), 
                                      textprops=dict(color="w"))

    ax.legend(wedges, labels,
              title= legend_title,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=10, weight="bold")
    plt.show()

In [ ]:
label = ['positive', 'negative',]
count_data = [total_positif+1, total_negatif+1,]

show_pie(label, count_data, "TextBlob")

In [ ]:
from wordcloud import WordCloud, STOPWORDS

def plot_cloud(wordcloud):
    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud) 
    plt.axis("off");

In [ ]:
all_words = ' '.join([tweets for tweets in df['Text']])
wordcloud = WordCloud(width = 3000, height = 2000, random_state=3, background_color='white', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(all_words)
plot_cloud(wordcloud)

In [ ]:
df2.to_csv("Data_Klasifikasi.csv")

In [ ]:
df2.to_csv("Data_Klasifikasi.xlsx")